In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Cargamos los datos del CSV que subiste
df = pd.read_csv('SP1.csv')

# Eliminamos valores vacíos para garantizar la robustez del modelo
df = df.dropna(subset=['HomeTeam', 'AwayTeam', 'FTR', 'HST', 'AST', 'HC', 'AC'])

# Seleccionamos nuestras "Features" (X) y lo que queremos predecir (y)
X = df[['HST', 'AST', 'HC', 'AC']] # Tiros a puerta y córners
y = df['FTR']                      # Resultado final (H, D, A)

# 1. Dividimos los datos: 80% para entrenar y 20% para probar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Creamos el escalador (Normalización)
scaler = StandardScaler()

# 3. Ajustamos el escalador con los datos de entrenamiento y transformamos ambos grupos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("¡Datos normalizados y listos para el modelo!")

# 1. Creamos la instancia del modelo de Regresión Logística
modelo_futbol = LogisticRegression(multi_class='ovr', solver='liblinear')

# 2. Entrenamos el modelo con los datos que ya escalamos
modelo_futbol.fit(X_train_scaled, y_train)

# 3. Hacemos las predicciones con el grupo de prueba (el 20% que reservamos)
predicciones = modelo_futbol.predict(X_test_scaled)

print("¡Modelo entrenado y predicciones generadas!")

# Mostramos las métricas de rendimiento
print("--- Informe de Clasificación ---")
print(classification_report(y_test, predicciones))

# También calculamos la precisión total (accuracy)
print(f"Precisión total del modelo: {accuracy_score(y_test, predicciones):.2f}")

from sklearn.model_selection import cross_val_score

# Realizamos una validación cruzada de 5 pliegues (cv=5)
# Usamos X_scaled (todos los datos normalizados) e y
scores = cross_val_score(modelo_futbol, scaler.fit_transform(X), y, cv=5)

print(f"Resultados de cada 'fold': {scores}")
print(f"Precisión media: {scores.mean():.2f}")

¡Datos normalizados y listos para el modelo!
¡Modelo entrenado y predicciones generadas!
--- Informe de Clasificación ---
              precision    recall  f1-score   support

           A       0.64      0.58      0.61        24
           D       0.38      0.14      0.21        21
           H       0.48      0.71      0.57        31

    accuracy                           0.51        76
   macro avg       0.50      0.48      0.46        76
weighted avg       0.50      0.51      0.48        76

Precisión total del modelo: 0.51
Resultados de cada 'fold': [0.60526316 0.55263158 0.52631579 0.60526316 0.51315789]
Precisión media: 0.56


c:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site